<a href="https://colab.research.google.com/github/tathagata-raha/nlp_project/blob/master/Project_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install conll-df

  ERROR: Failed building wheel for conll-df
  Running setup.py clean for conll-df
Failed to build conll-df
    Running setup.py install for conll-df ... done


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences


Using TensorFlow backend.


In [0]:
import pandas as pd
from conll_df import conll_df
path = 'hind.conllu'
df = conll_df(path, file_index=False)

In [0]:
curr = 1
arr =  []
newArr = []
sentArray = []
tagArray = []
sentnew =[]
tagnew=[]
for i in df.itertuples(): 
  if(curr>1000):
    break
  if(i[0][0]>curr):
    #print(curr)
    arr.append([j for j in newArr])
    sentArray.append([j for j in sentnew])
    tagArray.append([j for j in tagnew])
    curr+=1
    newArr = []
  else:
    newArr.append((i[1],i[3]))
    sentnew.append(i[1])
    tagnew.append(i[3])



In [0]:
(train_sentences, 
 test_sentences, 
 train_tags, 
 test_tags) = train_test_split(sentArray, tagArray, test_size=0.2)
 


In [0]:
words, tags = set([]), set([])
 
for s in train_sentences:
    for w in s:
        words.add(w.lower())
 
for ts in train_tags:
    for t in ts:
        tags.add(t)
 
word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0  # The special value used for padding
word2index['-OOV-'] = 1  # The special value used for OOVs
 
tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0  # The special value used to padding
 

In [7]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []
 
for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    train_sentences_X.append(s_int)
 
for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
 
    test_sentences_X.append(s_int)
 
for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])
 
for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[3224, 3247, 1388, 2335, 2204, 2682, 2312, 1804, 2522, 3715, 3667, 70, 2978, 3455, 3489, 3379, 3667, 2105, 747, 1027, 479, 3715, 3667, 506, 2576, 2321, 1067, 2467, 3294, 2911, 2312, 1150, 2247, 2502, 1796, 2105, 747, 2321, 3667, 329, 2745, 3173, 3715, 3667, 782, 606, 47, 3199, 1528, 2625, 1152, 387, 2247, 2960, 2409, 2338, 3244, 3715, 3667, 770, 2247, 2408, 2204, 2484, 2247, 2067, 3674, 2879, 1528, 3224, 3173, 2347, 3648, 934, 2247, 3283, 1528, 3715, 2068, 3203, 934, 1722, 1504, 918, 3379, 3667, 179, 1067, 551, 1947, 1528, 1738, 1020, 3715, 3667, 242, 1388, 1107, 1993, 2079, 3715, 3667, 3459, 1804, 3224, 258, 1947, 1067, 884, 1947, 1528, 2202, 3715, 3667, 305, 1545, 3455, 969, 3095, 3715, 3667, 1301, 2312, 819, 1220, 1613, 2247, 795, 1898, 1798, 1528, 119, 2919, 2052, 1067, 1722, 3654, 1761, 3715, 3667, 2332, 2522, 1947, 1807, 1317, 1947, 3535, 1317, 2739, 2181, 2247, 2502, 2860, 1317, 2522, 1475, 1317, 885, 1067, 409, 3246, 1317, 1439, 3127, 1317, 2178, 536, 1528, 279, 1317, 397, 1067

In [8]:

MAX_LENGTH = len(max(train_sentences_X, key=len))
print(MAX_LENGTH) 

16562


In [9]:
train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LENGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LENGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LENGTH, padding='post')
 
print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])


[3224 3247 1388 ...    0    0    0]
[3224 3247 1388 ...    0    0    0]
[10 13  1 ...  0  0  0]
[10 13  1 ...  0  0  0]


In [10]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation, SimpleRNN
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(SimpleRNN(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 16562, 128)        476288    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 16562, 256)        98560     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16562, 16)         4112      
_________________________________________________________________
activation_1 (Activation)    (None, 16562, 16)         0         
Total params: 578,960
Trainable params: 578,960
Non-trainable params: 0
_________________________________________________________________


In [0]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [12]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [14]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=30, epochs=2, validation_split=0.2)


Train on 640 samples, validate on 160 samples
Epoch 1/2
640/640 [==============================] - 836s 1s/step - loss: 1.7998 - accuracy: 0.5424 - val_loss: 1.8080 - val_accuracy: 0.5968
Epoch 2/2
640/640 [==============================] - 843s 1s/step - loss: 1.1916 - accuracy: 0.6475 - val_loss: 1.0270 - val_accuracy: 0.6692


In [15]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]}: {scores[1] * 100}")   
 

200/200 [==============================] - 9s 47ms/step
accuracy: 68.15435886383057


In [16]:
test_samples = [
    "रामायण काल में भगवान राम के पुत्र कुश की राजधानी कुशावती को 483 ईसा पूर्व बुद्ध ने अपने अंतिम विश्राम के लिए चुना ।".split(),
    "मल्‍लों की राजधानी होने के कारण प्राचीनकाल में इस स्‍थान का अत्‍यंत महत्‍व था ।".split()
]
print(test_samples)

[['रामायण', 'काल', 'में', 'भगवान', 'राम', 'के', 'पुत्र', 'कुश', 'की', 'राजधानी', 'कुशावती', 'को', '483', 'ईसा', 'पूर्व', 'बुद्ध', 'ने', 'अपने', 'अंतिम', 'विश्राम', 'के', 'लिए', 'चुना', '।'], ['मल्\u200dलों', 'की', 'राजधानी', 'होने', 'के', 'कारण', 'प्राचीनकाल', 'में', 'इस', 'स्\u200dथान', 'का', 'अत्\u200dयंत', 'महत्\u200dव', 'था', '।']]


In [17]:
test_samples_X = []
for s in test_samples:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])
    test_samples_X.append(s_int)
 
test_samples_X = pad_sequences(test_samples_X, maxlen=MAX_LENGTH, padding='post')
print(test_samples_X)
 

[[   1 3283 2312 ...    0    0    0]
 [   1 1388 1155 ...    0    0    0]]


In [18]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

[[[6.6466674e-02 6.7244589e-02 6.3551962e-02 ... 6.2625714e-02
   6.0462907e-02 6.1793908e-02]
  [6.2392615e-02 7.2676815e-02 6.2002596e-02 ... 6.6422850e-02
   5.8587886e-02 6.0213182e-02]
  [5.2948102e-02 8.2279041e-02 6.8244249e-02 ... 6.2757090e-02
   5.7742570e-02 5.8850978e-02]
  ...
  [9.8061144e-01 1.4696787e-04 2.4642530e-03 ... 4.8810453e-04
   3.6461139e-03 5.8334536e-04]
  [9.8061144e-01 1.4696817e-04 2.4642565e-03 ... 4.8810543e-04
   3.6461188e-03 5.8334612e-04]
  [9.8061144e-01 1.4696787e-04 2.4642539e-03 ... 4.8810476e-04
   3.6461155e-03 5.8334565e-04]]

 [[6.6466674e-02 6.7244589e-02 6.3551962e-02 ... 6.2625714e-02
   6.0462907e-02 6.1793908e-02]
  [6.1190061e-02 8.8446535e-02 6.3587457e-02 ... 6.4523257e-02
   5.9632692e-02 6.1411206e-02]
  [5.0532024e-02 7.4762218e-02 6.2713459e-02 ... 6.5204158e-02
   5.1264886e-02 6.6730589e-02]
  ...
  [9.8061144e-01 1.4696801e-04 2.4642539e-03 ... 4.8810476e-04
   3.6461155e-03 5.8334565e-04]
  [9.8061144e-01 1.4696817e-04 2.464

In [0]:
def logits_to_tokens(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
            token_sequence.append(index[np.argmax(categorical)])
 
        token_sequences.append(token_sequence)
 
    return token_sequences

In [20]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))


[['NOUN', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADP', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'ADP', 'NOUN', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'ADP', 'PRON', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PAD-', '-PA

Making accuracy metric to ignore padding tokens 

In [0]:

from keras import backend as K
 
def ignore_class_accuracy(to_ignore=0):
    def ignore_accuracy(y_true, y_pred):
        y_true_class = K.argmax(y_true, axis=-1)
        y_pred_class = K.argmax(y_pred, axis=-1)
 
        ignore_mask = K.cast(K.not_equal(y_pred_class, to_ignore), 'int32')
        matches = K.cast(K.equal(y_true_class, y_pred_class), 'int32') * ignore_mask
        accuracy = K.sum(matches) / K.maximum(K.sum(ignore_mask), 1)
        return accuracy
    return ignore_accuracy
 

In [23]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation, SimpleRNN
from keras.optimizers import Adam
 
 
model = Sequential()
model.add(InputLayer(input_shape=(MAX_LENGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(SimpleRNN(256, return_sequences=True))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy', ignore_class_accuracy(0)])
 
model.summary()
 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 16562, 128)        476288    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 16562, 256)        98560     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 16562, 16)         4112      
_________________________________________________________________
activation_2 (Activation)    (None, 16562, 16)         0         
Total params: 578,960
Trainable params: 578,960
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=30, epochs=3, validation_split=0.2)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 640 samples, validate on 160 samples
Epoch 1/3
 90/640 [===>..........................] - ETA: 11:42 - loss: 2.4028 - accuracy: 0.4079 - ignore_accuracy: 0.1490

In [0]:
predictions = model.predict(test_samples_X)
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))